In [2]:
from numpy.lib.stride_tricks import as_strided
import numpy as np

In [ ]:
k_diags = np.array([0.25 * np.array([[-1, 0, 0, 0],
                                    [0, 1, 0, 0],
                                    [0, 0, 1, 0],
                                    [0, 0, 0, -1]]),
                    0.25 * np.array([[0, 0, 0, -1],
                                    [0, 0, 1, 0],
                                    [0, 1, 0, 0],
                                    [-1, 0, 0, 0]])])

k_lines = [0.25 * np.array([[-1, 1, 1, -1]]), 0.25 * np.array([[-1],[1],[1],[-1]])]

In [ ]:
arr = np.zeros((10, 10))

In [ ]:
arr[1][2] = 1
arr[2][2] = -1
arr[3][2] = -1
arr[4][2] = 1

arr[5][5] = 1
arr[5][6] = -1
arr[5][7] = -1
arr[5][8] = 1

arr[6][0] = 1
arr[7][1] = -1
arr[8][2] = -1
arr[9][3] = 1


arr

In [ ]:
sub_shape = k_diags[0].shape
print(sub_shape)

In [ ]:
view_shape = tuple(np.subtract(arr.shape, sub_shape) + 1) + sub_shape

In [ ]:
np.subtract(arr.shape, sub_shape) + 1

In [ ]:
view_shape

In [ ]:
arr_view = as_strided(arr, view_shape, arr.strides * 2)

In [ ]:
arr.strides * 2

In [ ]:
arr_view

In [ ]:
arr_view = arr_view.reshape((-1,) + sub_shape)
arr_view

In [ ]:
res = np.multiply(arr_view, k_diags[0])

In [ ]:
res

In [ ]:
res.shape

In [ ]:
np.einsum('ijk->i', res)

In [ ]:
arr_test = np.zeros((10,10))

In [ ]:
arr_test[2][8] = 2

In [ ]:
arr_res = np.argwhere(arr_test > 3)

In [ ]:
arr_res.shape[0] == 0

In [ ]:
arr_res[0]

In [ ]:
-1 * 0.25 * np.array([-1, 1, 1, -1])

In [ ]:
k_vertical = 0.25 * np.array([[-1], [1], [1], [-1]])
k_horizontal = 0.25 * np.array([[-1], [1], [1], [-1]])

In [ ]:
arr_test[0][1] = 1
arr_test[1][1] = -1
arr_test[2][1] = -1
arr_test[3][1] = 1
arr_test

In [95]:
k_diags = np.array([0.25 * np.array([[-1, 0, 0, 0],
                                    [0, 1, 0, 0],
                                    [0, 0, 1, 0],
                                    [0, 0, 0, -1]]),
                    0.25 * np.array([[0, 0, 0, -1],
                                    [0, 0, 1, 0],
                                    [0, 1, 0, 0],
                                    [-1, 0, 0, 0]])])

k_lines = [0.25 * np.array([[-1, 1, 1, -1]]), 0.25 * np.array([[-1],[1],[1],[-1]])]

def _subboard_4_Conv2D(grid, k_shape:tuple, stride:tuple) -> np.array:
    """ Generates the sub view of the grid to be multiply with the kernel.
    First the shape of the sub_grid array is calculated, it depends on
    the grid shape and the kernel shape.
    The sub_grid array shape will be (n_x, n_y, k_x, k_y) with:
        * n_x: number of application of the kernel along row (with stride of 1)
        * n_y: number of application of the kernel along column (with stride of 1)
        * k_x, k_y: the shape of the kernel
    In this way sub_grid is a numpy array of n_x/n_y rows/columns of (k_x x k_y)
    sub view of the grid.
    Args:
    -----
        k_shape ([tuple[int]]): shape of the kernel
        stride ([tuple(int)]): put self.grid.strides * 2 (but why?)
    """
    view_shape = tuple(np.subtract(grid.shape, k_shape) + 1) + k_shape
    sub_grid = as_strided(grid, view_shape, stride * 2)
    return sub_grid


def _my_conv2D(grid, kernel:np.array) -> np.array:
    """ Retrieves the sub_grid from the function _subboard_4_Conv2D and performs
    the convolution (array multiplication + einstein sum along the 3rd and 4th
    dimensions).
    Args:
    -----
        * kernel ([np.array]): the kernel to use for convolution.
    """
    sub_grid = _subboard_4_Conv2D(grid, k_shape=kernel.shape, stride=grid.strides)
    res_conv = np.multiply(sub_grid, kernel)
    convolved = np.einsum('ijkl->ij', res_conv)
    return convolved.astype('int8')


def check_board(grid):
    """[summary]
    """
    ## Checking if white pair captured
    # Checking the diagonal:
    conv_diag1 = _my_conv2D(k_diags[0])
    conv_diag2 = _my_conv2D(k_diags[1])
    # Checking vertical and horizontal
    conv_lin1 = _my_conv2D(k_lines[0])
    conv_lin2 = _my_conv2D(k_lines[1])
    
    coord_cd1 = np.argwhere(conv_diag1 == 1)
    coord_cd2 = np.argwhere(conv_diag2 == 1)
    coord_cl1 = np.argwhere(conv_lin1 == 1)
    coord_cl2 = np.argwhere(conv_lin2 == 1)
    if coord_cd1.shape[0] != 0:
        for coord in coord_cd1:
            print("[check_board] - conv_diag1")
            grid[coord[0] + 1][coord[1] + 1] = 0
            grid[coord[0] + 2][coord[1] + 2] = 0
    if coord_cd2.shape[0] != 0:
        for coord in coord_cd2:
            print("[check_board] - conv_diag2")
            grid[coord[0] + 1][coord[1] + 2] = 0
            grid[coord[0] + 2][coord[1] + 1] = 0
    if coord_cl1.shape[0] != 0:
        for coord in coord_cl1:
            print("[check_board] - conv_lin1")
            grid[coord[0]][coord[1] + 1] = 0
            grid[coord[0]][coord[1] + 2] = 0
    if coord_cl2.shape[0] != 0:
        for coord in coord_cl2:
            print("[check_board] - conv_lin2")
            grid[coord[0] + 1][coord[1]] = 0
            grid[coord[0] + 2][coord[1]] = 0
    ## Checking if black pair captured
    # Checking the diagonal:
    conv_diag1 = _my_conv2D(-1 * k_diags[0])
    conv_diag2 = _my_conv2D(-1 * k_diags[1])
    # Checking vertical and horizontal
    conv_lin1 = _my_conv2D(-1 * k_lines[0])
    conv_lin2 = _my_conv2D(-1 * k_lines[1])
    print("<<<== CONV_LIN1 ==>>>")
    print(conv_lin1)
    print("<<<== CONV_LIN2 ==>>>")
    print(conv_lin2)
    print("<<<== ========= ==>>>")
    coord_cd1 = np.argwhere(conv_diag1 == 1)
    coord_cd2 = np.argwhere(conv_diag2 == 1)
    coord_cl1 = np.argwhere(conv_lin1 == 1)
    coord_cl2 = np.argwhere(conv_lin2 == 1)
    if coord_cd1.shape[0] != 0:
        for coord in coord_cd1:
            print("[check_board] - conv_diag1 -1")
            grid[coord[0] + 1][coord[1] + 1] = 0
            grid[coord[0] + 2][coord[1] + 2] = 0
    if coord_cd2.shape[0] != 0:
        for coord in coord_cd2:
            print("[check_board] - conv_diag2 -1")
            grid[coord[0] + 1][coord[1] + 2] = 0
            grid[coord[0] + 2][coord[1] + 1] = 0
    if coord_cl1.shape[0] != 0:
        for coord in coord_cl1:
            print("[check_board] - conv_lin1 -1")
            grid[coord[0]][coord[1] + 1] = 0
            grid[coord[0]][coord[1] + 2] = 0
    if coord_cl2.shape[0] != 0:
        for coord in coord_cl2:
            print("[check_board] - conv_lin2 -1")
            grid[coord[0] + 1][coord[1]] = 0
            grid[coord[0] + 2][coord[1]] = 0

In [38]:
_subboard_4_Conv2D(arr_test, k_vertical, arr_test.strides)

NameError: name 'arr_test' is not defined

In [ ]:
view_shape = tuple(np.subtract(arr_test.shape, k_vertical.shape) + 1) + k_vertical.shape

In [ ]:
view_shape

In [ ]:
sub_grid = as_strided(arr_test, view_shape, arr_test.strides * 2)


In [ ]:
sub_grid

In [ ]:
res_conv = np.multiply(sub_grid, k_vertical)


In [ ]:
res_conv

In [ ]:
convolved = np.einsum('ijkl->kl', res_conv)

In [ ]:
convolved

In [193]:
def issimplefreethree_position(yx, grid):
    """[summary]
    Args:
    yx ([type]): [description]
    grid ([type]): [description]
    Returns:
    ___: [description]
    """
    tmp = np.zeros((18,18))
    tmp[4:-4, 4:-4] = grid + 1 
    r_start, r_end = yx[0], yx[0] + 8
    c_start, c_end = yx[1], yx[1] + 8
    print(tmp[r_start:r_end, c_start:c_end])
    res = [_my_conv2D(tmp[r_start:r_end, c_start:c_end], kernel) for kernel in k_free_threes]
    if any([np.any(arr == 15) for arr in res]):
        return True, res
    return False, res
            

In [222]:
grid1 = np.zeros((10,10))
grid2 = np.zeros((10,10))
grid3 = np.zeros((10,10))
grid4 = np.zeros((10,10))
grid5 = np.zeros((10,10))
grid6 = np.zeros((10,10))
grid7 = np.zeros((10,10))

In [217]:
#k_free_threes = [np.ones((1,6)), np.ones((6,1)), np.identity(6), np.rot90(np.identity(6))]
k_free_threes = [np.array([[1, 2, 2, 2, 2, 1]]),
                 np.array([[1], [2], [2], [2], [2], [1]]),
                 np.array([[1, 0, 0, 0, 0, 0],
                           [0, 2, 0, 0, 0, 0],
                           [0, 0, 2, 0, 0, 0],
                           [0, 0, 0, 2, 0, 0],
                           [0, 0, 0, 0, 2, 0],
                           [0, 0, 0, 0, 0, 1]]),
                 np.array([[0, 0, 0, 0, 0, 1],
                           [0, 0, 0, 0, 2, 0],
                           [0, 0, 0, 2, 0, 0],
                           [0, 0, 2, 0, 0, 0],
                           [0, 2, 0, 0, 0, 0],
                           [1, 0, 0, 0, 0, 0]])]

In [223]:
grid1[0:3,2] = 1

grid2[1:4,2] = 1

grid3[5,0:3] = 1

grid4[6,2:5] = 1

grid5[2,2] = 1
grid5[3,3] = 1
grid5[4,4] = 1

grid6[2][2] = 1
grid6[4][4] = 1
grid6[5][5] = 1

grid7[2][2] = 1
grid7[3][3] = 1
grid7[5][5] = 1


In [236]:
grid4

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 1., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [251]:
_, res = issimplefreethree_position([6,2], grid4)

[[0. 0. 1. 1. 1. 1. 1. 1.]
 [0. 0. 1. 1. 1. 1. 1. 1.]
 [0. 0. 1. 1. 1. 1. 1. 1.]
 [0. 0. 1. 1. 1. 1. 1. 1.]
 [0. 0. 1. 1. 2. 2. 2. 1.]
 [0. 0. 1. 1. 1. 1. 1. 1.]
 [0. 0. 1. 1. 1. 1. 1. 1.]
 [0. 0. 1. 1. 1. 1. 1. 1.]]


In [252]:
res

[array([[ 7,  9, 10],
        [ 7,  9, 10],
        [ 7,  9, 10],
        [ 7,  9, 10],
        [10, 14, 16],
        [ 7,  9, 10],
        [ 7,  9, 10],
        [ 7,  9, 10]], dtype=int8),
 array([[ 0,  0, 10, 10, 12, 12, 12, 10],
        [ 0,  0, 10, 10, 12, 12, 12, 10],
        [ 0,  0, 10, 10, 12, 12, 12, 10]], dtype=int8),
 array([[ 9, 11, 12],
        [ 7, 11, 12],
        [ 7,  9, 12]], dtype=int8),
 array([[ 7,  9, 10],
        [ 7,  9, 12],
        [ 7, 11, 12]], dtype=int8)]